#### Task1
В качестве разрешенных выбраны следующие кодовые комбинации:<br>
  - *Е1=0000110111*
  - *Е2=0000101011*
  - *Е3=1000011001*
  - *Е4=0010010110*

Определить максимальную кратность обнаруживаемых и исправляемых ошибок.<br>
Показать процесс исправления ошибок максимальной кратности при передаче первой
разрешенной кодовой комбинации.

In [140]:
import numpy as np
from random import randrange

def xor(a, b):
    return (a & (~b)) | ((~a) & b)

# messages with length of 7
msg_len = 10;
E1 = 0b1000101011
E2 = 0b0100110110
E3 = 0b0010111001
E4 = 0b0001110111

# list of messages
msg_list = np.array([0, E1, E2, E3, E4])

msg_list_len = len(msg_list)

# dest array
d = np.array([[0] * msg_list_len] * msg_list_len)

d_list = []

# calc dests
for i in range(1, msg_list_len - 1):
    for j in range(i + 1, msg_list_len):
        d[i][j] = xor(msg_list[i], msg_list[j]).bit_count()
        print(f"d{i}{j} = {d[i][j]}")
        d_list.append(d[i][j])

d_min = min(d_list)
print(f"\nd_min = {d_min}")


# t_ob -- order of error that is able to be detected
print("\nd_min >= t_ob + 1")
t_ob = d_min - 1
print(f"t_ob <= {t_ob}\n")


# t_isp -- order of error that is able to be fixed
print("d_min >= 2*t_isp + 1")
t_isp = (d_min - 1) / 2
print(f"t_isp <= {t_isp}\n")


# any error vector of t_isp order
# error_vec = 0b0 | (1 << randrange(msg_len))
error_vec = 0b1000000000
print(f"error_vec = {bin(error_vec)}\n")


# adding error_vec in msg E1
new_E1 = xor(E1, error_vec)
print(f"new_E1 = E1 ^ error_vec = {bin(new_E1)}")


# check dest from new msg to start msgs
min_dest = msg_len + 1
min_dest_i = 0
for i in range(1, msg_list_len):
    dest = xor(msg_list[i], new_E1).bit_count()
    print(f"d{i} = {dest}")

    if dest < min_dest :
        min_dest = dest
        min_dest_i = i

print(f"min_dest = d{min_dest_i} = {min_dest}\n")

if 1 == min_dest_i:
    print(f"Closest msg is E{min_dest_i}")
    print("So it's E1")

d12 = 6
d13 = 4
d14 = 5
d23 = 6
d24 = 3
d34 = 5

d_min = 3

d_min >= t_ob + 1
t_ob <= 2

d_min >= 2*t_isp + 1
t_isp <= 1.0

error_vec = 0b1000000000

new_E1 = E1 ^ error_vec = 0b101011
d1 = 1
d2 = 5
d3 = 3
d4 = 4
min_dest = d1 = 1

Closest msg is E1
So it's E1


#### Task 2

Построить линейный код, обеспечивающий исправление одиночных ошибок при передаче
*N=76<sub>10</sub>* сообщений.<br>
Закодировать информацию *И=53<sub>8</sub>* и при действии вектора ошибок
*е1=1000<sub>2</sub>* показать процесс обнаружения и исправления одиночной ошибки.

In [141]:
import math as mt
import numpy as np

msg_num = 76
e1 = [1, 0, 0, 0]
hex_info = 0x53
info = [int(i) for i in list('{0:0b}'.format(hex_info))]

print(info)

# order of error that is able to be fixed
t_isp = 1

print(f"msg_num = {msg_num}")
print(f"e1 = {e1}")
print(f"info = {hex(hex_info)}")
print(f"t_isp = {t_isp}")

info_length = mt.ceil(mt.log2(msg_num))
print(f"\ninfo_length = {info_length}")

crc_length = 0

# sum_length = crc_length + info_length
while pow(2, crc_length) < info_length + crc_length + 1 :
    crc_length+=1

print("\n2^crc_l >= crc_l + info_l + 1")
print(f"crc_lentgh = {crc_length}")

# d_min >= 2 * t_isp + 1
d_min = 2 * t_isp + 1

print(f"\nd_min >= 2 * t + 1 = {d_min}\n")

info_matrix = np.identity(info_length)

crc_matrix = np.matrix([[0, 0, 1, 1],
                        [0, 1, 1, 0],
                        [1, 1, 0, 0],
                        [1, 0, 0, 1],
                        [1, 1, 0, 1],
                        [1, 1, 1, 1],
                        [1, 0, 1, 0]])

full_matrix = np.concatenate((info_matrix, crc_matrix), axis=1)

print(full_matrix)

for i in range (len(info), info_length) :
    info.insert(0, 0)

for i in range (len(e1), info_length) :
    e1.insert(0, 0)

print(f"exteneded info = {info}\n")

e1_matrix = np.matrix([e1])
info_vector = np.matrix([info])

E = info_vector.dot(full_matrix) % 2

print(f"E = {E}")



[1, 0, 1, 0, 0, 1, 1]
msg_num = 76
e1 = [1, 0, 0, 0]
info = 0x53
t_isp = 1

info_length = 7

2^crc_l >= crc_l + info_l + 1
crc_lentgh = 4

d_min >= 2 * t + 1 = 3

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]
 [0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0.]]
exteneded info = [1, 0, 1, 0, 0, 1, 1]

E = [[1. 0. 1. 0. 0. 1. 1. 1. 0. 1. 0.]]
